In [19]:
import os
import xml.etree.ElementTree as ET
from utils.finder import find_leafes
from utils.dicom_to_nifti import seperate_labels

In [20]:
directory = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)'
annotation_dir = r'C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Detail annotaties\Detail Vaat(-)'
#boundingbox_dir = r"C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Bounding boxes\BB Tumor(+), Vaat(+)"
output = r"C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\Processed_Data"

In [21]:
par_list = ['par', 'pan', '40']
pvp_list = ['ven', "pvp", "80", "port", "abdomen", "th abd ax 2 2", "tho+abd"]
art_list = ['art']
liver_list = ['laat']
ct_list = ['ct', 'tho', 'thx', 'buik']
patient_cases = {}

In [22]:
for case in os.listdir(directory):
    
    if os.path.isdir(os.path.join(directory, case)):
        case_number = case

        if int(case_number) != 1075:
            continue

        sectra = os.path.join(directory, case, "SECTRA", "CONTENT.XML")
        modalities = []
        if (os.path.exists(sectra)):

            root = ET.parse(sectra).getroot()
            print("==========================")
            print(case)
            print("==========================")

            for series in root.findall('patient/request/study/series'):
                for description in series.findall('series_data/description'):
                    #print(series)
                    id = series.get('id')
                    print(f"ID: {id}")
                    print(description.text)
                    phase = None

                    if description.text is not None:
                       
                        
                        if bool([ele for ele in par_list if(ele in description.text.lower())]):
                            print("PARENCHYMA")
                            phase = "par"
                        elif bool([ele for ele in pvp_list if(ele in description.text.lower())]):
                            print("PVP")
                            phase = "pvp"
                        elif bool([ele for ele in art_list if(ele in description.text.lower())]):
                            print("ART")
                            phase = "art"
                        elif bool([ele for ele in liver_list if(ele in description.text.lower())]):
                            print("liver")
                            phase = "liver"
                        else:
                            phase = "unknown"

                    else:
                        phase = "unknown"

                    modalities.append(phase)

                    if (phase == "unknown"):
                        print("Failed to find phase")
                        continue

                    high_data_dir = os.path.join(directory, case_number)
                    low_dir = find_leafes(high_data_dir)
                    data_dir = os.path.dirname(low_dir)

                    spec_data_dar = os.path.join(data_dir, id)
                    print(phase)
                    high_annotation_directory = os.path.join(annotation_dir, case_number, phase)
                   
                    spec_annotation_directory = find_leafes(high_annotation_directory)
                   
                    
                    if os.path.exists(spec_data_dar):
                        print(f"Data dir {spec_data_dar} found")
                    else:
                         print(f"Data dir {spec_data_dar} does not exist")
                         continue

                    if os.path.exists(spec_annotation_directory):
                        print(f"Annotation dir {spec_annotation_directory} found")
                    else:
                         print(f"Annotation dir {spec_annotation_directory} does not exist")
                         continue

                    
                    seperate_labels(spec_data_dar, spec_annotation_directory, output, case_number, phase)
                    
        else:

            print(f"{case} has an error")
            print(f"{sectra} does not exist")
            modalities.append("no info")
        patient_cases[case_number] = modalities


1075
ID: 00009D19
PARENCHYM 1/1
PARENCHYMA
par
Data dir C:\Users\320088652\OneDrive - Philips\TUe\eMTIC\Pancreas_BB\Pancreas_BB\CT scans\CT scans Vaat (-)\1075\DICOM\0000AEB3\AA77E4F6\AA84D56F\00009D19 found


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType